# Vital Parks Script Part 1

## This is the part of the vital parks script that builds the travel network and stores it. 

# TO DO! 
Add Ferries. Add City Bike. Add park paths. Add driving?

# Set Parameters

In [1]:
# This is the assigned walk speed in miles per hr
walk_speed_miles_per_hr = 3.0

# If taking transit we take the average transit travel times on weekdays by default.
# If weekend_transit=True then we take the average transit travel times on weekends.
weekend_transit = False

# If taking transit we take the average transit travel times during these hours (24 hr clock)
transit_start_hr = 7
transit_end_hr = 18

# Import Packages

In [ ]:
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import requests

from utils.make_network import (
    create_census_geoid,
    gtfs_mta_import,
    format_mta_stops_data,
)

# PREPARE STREET AND TRANSIT NETWORKS

## Prepare street network

### Pull and modify LION data

In [ ]:
# Read in lion data. It can only be read in 4000 row batches so we pull the entire dataset with a while loop
not_finished = True
max_object_id = 0
data = {}
i = 0
url = """https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/LION/FeatureServer/0/query"""
while not_finished:
    params = {
        "where": """(FeatureTyp IN ('0','6','A','C','W'))AND(NodeLevelF <> '*')AND(NodeLevelT <> '*')AND(NonPed <> 'V')AND(RW_TYPE <>' 2')AND(RW_TYPE <>'2')AND(RW_TYPE <>'11')AND(RW_TYPE <>'12')AND(RW_TYPE <>'13')AND(RW_TYPE <>'14')AND(OBJECTID>{})""".format(
            max_object_id
        ),
        "outfields": """OBJECTID,FeatureTyp,Street,SegmentID,RW_TYPE,NYPDID,FromLeft,ToLeft,FromRight,ToRight,XFrom,YFrom,XTo,YTo,NodeIDFrom,NodeIDTo,NodeLevelF,NodeLevelT,LBoro,RBoro,L_CD,R_CD,LCT2020,RCT2020,LCT2020Suf,RCT2020Suf,LCB2020,RCB2020,LCB2020Suf,RCB2020Suf""",
        "outSR": "4326",
        "limit": "4000",
        "f": "json",
    }
    response = requests.get(url, params=params)
    data_current = response.json()
    data[i] = pd.json_normalize(data_current["features"])
    max_object_id = data_current["features"][-1]["attributes"]["OBJECTID"]
    if len(data[i]) < 4000:
        not_finished = False
    i = i + 1
lion = pd.concat(data).reset_index(drop=True)
lion.columns = lion.columns.str.lstrip("attributes.")
lion = lion.rename(columns={"geometry.paths": "geometry"})
lion["geometry"] = lion["geometry"].apply(lambda x: LineString(x[0]))
lion = gpd.GeoDataFrame(lion, geometry="geometry", crs=4326).to_crs(2263)

# Make sure source and target nodes include node-level as part of its unique ID so we keep track of how roads actually connect accounting for both location and level
lion["source"] = lion["NodeIDFrom"] + lion["NodeLevelF"]
lion["target"] = lion["NodeIDTo"] + lion["NodeLevelT"]

# Change names of From/To so it matches our Source/Target vocabulary as we keep track of lat/lon coordinates for each node
lion = lion.rename(
    columns={
        "XFrom": "XCoord_source",
        "YFrom": "YCoord_source",
        "XTo": "XCoord_target",
        "YTo": "YCoord_target",
    }
)

In [ ]:
# Convert walk speed to feet since we will be using crs=2263 for this script which measures distance in feet
walk_speed_feet_per_second = walk_speed_miles_per_hr * 1.46667

# Group by segment ID so segmentID becomes a unique ID.
# This is needed because there are times when multiple rows have the same SegmentID but in these cases they are actually representing the same physical segment
# These duplications exist for to other Lion dataset uses that we do not need for this instance.
lion_walkable = (
    lion.groupby("SegmentID")
    .agg(
        {
            "source": "min",
            "target": "max",
            "Street": "first",
            "FeatureTyp": "min",
            "RW_TYPE": "min",
            "FromLeft": "min",
            "ToLeft": "max",
            "FromRight": "min",
            "ToRight": "max",
            "NYPDID": "first",
            "LBoro": "first",
            "RBoro": "first",
            "L_CD": "first",
            "R_CD": "first",
            "LCT2020": "first",
            "RCT2020": "first",
            "LCT2020Suf": "first",
            "RCT2020Suf": "first",
            "LCB2020": "first",
            "RCB2020": "first",
            "LCB2020Suf": "first",
            "RCB2020Suf": "first",
            "XCoord_source": "mean",
            "YCoord_source": "mean",
            "XCoord_target": "mean",
            "YCoord_target": "mean",
            "geometry": "first",
        }
    )
    .reset_index()
)
lion_walkable = gpd.GeoDataFrame(lion_walkable, geometry="geometry", crs=2263)

# Assign each segment a weight equal to the number of seconds required to travel (walk) each segment
lion_walkable["length"] = lion_walkable["geometry"].apply(lambda x: float(x.length))
lion_walkable["weight"] = lion_walkable["length"] / walk_speed_feet_per_second
lion_walkable["mode"] = "walk"

In [ ]:
# Split up each street centerline into its left/right block.
# Assign each block a unique idea by its segmentID + (L or R)

# Get left blocks from street centerline data
lion_walkable_left = lion_walkable[
    [
        "SegmentID",
        "Street",
        "source",
        "target",
        "weight",
        "mode",
        "FeatureTyp",
        "RW_TYPE",
        "FromLeft",
        "ToLeft",
        "length",
        "NYPDID",
        "LBoro",
        "L_CD",
        "LCT2020",
        "LCT2020Suf",
        "LCB2020",
        "LCB2020Suf",
        "XCoord_source",
        "YCoord_source",
        "XCoord_target",
        "YCoord_target",
        "geometry",
    ]
]
lion_walkable_left = lion_walkable_left.rename(
    columns={
        "FromLeft": "From",
        "ToLeft": "To",
        "LBoro": "Boro",
        "L_CD": "CommunityBoard",
        "LCT2020": "CT2020",
        "LCT2020Suf": "CT2020Suf",
        "LCB2020": "CB2020",
        "LCB2020Suf": "CB2020Suf",
    }
)
lion_walkable_left["SideOfStreet"] = "L"
lion_walkable_left["uniqueID"] = lion_walkable_left["SegmentID"] + "L"

# Get right blocks from street centerline data
# We flip the order of source/target nodes for the right blocks. This is just a way to make sure each street segment in our network can be walked in both directions.
lion_walkable_right = lion_walkable[
    [
        "SegmentID",
        "Street",
        "source",
        "target",
        "weight",
        "mode",
        "FeatureTyp",
        "RW_TYPE",
        "FromRight",
        "ToRight",
        "length",
        "NYPDID",
        "RBoro",
        "R_CD",
        "RCT2020",
        "RCT2020Suf",
        "RCB2020",
        "RCB2020Suf",
        "XCoord_source",
        "YCoord_source",
        "XCoord_target",
        "YCoord_target",
        "geometry",
    ]
]
lion_walkable_right = lion_walkable_right.rename(
    columns={
        "source": "target",
        "target": "source",
        "FromRight": "From",
        "ToRight": "To",
        "RBoro": "Boro",
        "R_CD": "CommunityBoard",
        "RCT2020": "CT2020",
        "RCT2020Suf": "CT2020Suf",
        "RCB2020": "CB2020",
        "RCB2020Suf": "CB2020Suf",
        "XCoord_source": "XCoord_target",
        "YCoord_source": "YCoord_target",
        "XCoord_target": "XCoord_source",
        "YCoord_target": "YCoord_source",
    }
)
lion_walkable_right["SideOfStreet"] = "R"
lion_walkable_right["uniqueID"] = lion_walkable_right["SegmentID"] + "R"

# Concat left and right blocks together to get all blocks throughout the city included in our walkable network
lion_walkable = pd.concat([lion_walkable_left, lion_walkable_right])

# For each source-target pair we make an edge label in the format of networkX as we will compare these edges to network edges soon.
lion_walkable["edges"] = list(zip(lion_walkable["source"], lion_walkable["target"]))

### Connect LION walkable streets to OSM park paths (To Do in future version)

### Import lion data into networkx data structure so we can remove small disconnected areas that are not part of this analysis

In [ ]:
# Create a network graph in networkX from our lion_walkable dataframe
Walk_DG = nx.from_pandas_edgelist(lion_walkable, create_using=nx.DiGraph())

# Remove all but largest two connected components (Walkably connected NYC + Walkably connected Staten Island)
Walk_DG = Walk_DG.subgraph(
    [
        p
        for ps in sorted(
            list(nx.strongly_connected_components(Walk_DG)), key=len, reverse=True
        )[:2]
        for p in ps
    ]
)

# Restrict our lion_walkable table to only include edges in one of these two largest connected components
lion_walkable = lion_walkable[lion_walkable["edges"].isin(list(Walk_DG.edges))]
lion_walkable = lion_walkable.reset_index(drop=True)

### Match Lion Data to Census Data

In [ ]:
# there are a few lion streets that have no census geography info. We keep these streets for potential walking uses but will assign them no population.
lion_walkable.loc[lion_walkable["Boro"].isna(), "Boro"] = 0


# Create GEO_ID from each blocks geographical data so that we merge census data to city blocks
lion_walkable["Geo_ID"] = create_census_geoid(lion_walkable)


# Assign 0 'livable length' to blocks that have no addresses on them. We will then proportionally assign census block populations according to each city block's livable length.
lion_walkable["livable_length"] = lion_walkable["length"]
lion_walkable.loc[
    ((lion_walkable["From"] == 0) & (lion_walkable["To"] == 0)), "livable_length"
] = 0
lion_walkable.loc[lion_walkable["Geo_ID"].isna(), "livable_length"] = 0

In [ ]:
# Import and format 2020 census population data to join to lion dataset
url = """https://api.census.gov/data/2020/dec/dhc?get=group(P1)&for=block%20group:*&in=state:36&in=county:061,005,047,081,085&in=tract:*"""
response = requests.get(url)
Census_data = response.json()
Census_data = pd.DataFrame(Census_data[1:], columns=Census_data[0])
Census_data["GEO_ID"] = Census_data["GEO_ID"].str[-10:]
Census_data = Census_data[["GEO_ID", "P1_001N"]].rename(
    columns={"GEO_ID": "Geo_ID", "P1_001N": "BG_total_pop"}
)

In [ ]:
# Assign census-block population data to city blocks proportional to each city-blocks livable length

# Calculate the proportion of livable length that each city-block makes up for in its respective census-block-group
total_liveable_lengths = (
    lion_walkable[["Geo_ID", "livable_length"]]
    .groupby("Geo_ID")
    .sum()
    .reset_index()
    .rename(columns={"livable_length": "Total_geo_ID_livable_length"})
)
lion_walkable = lion_walkable.merge(total_liveable_lengths, how="left", on="Geo_ID")
lion_walkable.loc[:, "Proportion_of_blockgroup"] = lion_walkable["livable_length"] / (
    lion_walkable["Total_geo_ID_livable_length"] + 0.00001
)

# Merge with census data and assign population from each census block group to each city block according to the above calculated proportion
lion_walkable = lion_walkable.merge(Census_data, how="left", on="Geo_ID")
lion_walkable.loc[lion_walkable["BG_total_pop"].isna(), "BG_total_pop"] = 0
lion_walkable["BG_total_pop"] = lion_walkable["BG_total_pop"].astype(float)
lion_walkable["population"] = (
    lion_walkable["Proportion_of_blockgroup"] * lion_walkable["BG_total_pop"]
)

### Assign Lion streets to the geographical regions of interest that are not already in lion features data

#### Assign lion streets to City Council Districts

In [ ]:
# Import and formate council district boundaries

url = """https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_City_Council_Districts/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson"""
council_district_shapes = gpd.read_file(url, params=params)
council_district_shapes = council_district_shapes[["CounDist", "geometry"]].to_crs(2263)
council_district_shapes = council_district_shapes.rename(
    columns={"CounDist": "Council_District"}
)
council_district_shapes["Council_District"] = council_district_shapes[
    "Council_District"
].astype(int)

In [ ]:
# Assign city blocks to council districts based on geographic data. (This is an estimate and could be made more precise with some geographical research.)

council_near = lion_walkable.loc[:, ["uniqueID", "geometry"]].sjoin_nearest(
    council_district_shapes, how="left", distance_col="dis"
)
council_near = council_near.sort_values(
    "Council_District", ascending=False
).sort_values("dis", ascending=False)
council_near = council_near.groupby("uniqueID").first().reset_index()
council_near = council_near[["uniqueID", "Council_District"]]

lion_walkable = lion_walkable.merge(council_near, on="uniqueID", how="left")

#### Assign lion streets to yes/no state designated disadvantaged communities

In [ ]:
# Import and formate disadvantaged community census tracts

url = """https://data.ny.gov/resource/2e6c-s6fp.geojson?$limit=10000&$where=nyc_region='NYC' """
DAC_shapes = gpd.read_file(url)

DAC_shapes = DAC_shapes[["geoid", "dac_designation", "geometry"]].to_crs(2263)
DAC_shapes["geoid"] = DAC_shapes["geoid"].str[2:]
DAC_shapes = DAC_shapes.rename(
    columns={"geoid": "DAC_ID", "dac_designation": "DAC_Designation"}
)

DAC_shapes = DAC_shapes.dissolve(by="DAC_ID").reset_index()

##### First try to match on ID

In [ ]:
# Match most blocks to this set of DAC tracts based on their Geo_ID.

lion_walkable["DAC_ID"] = lion_walkable["Geo_ID"].str[:-1]
lion_walkable = lion_walkable.merge(
    DAC_shapes[["DAC_ID", "DAC_Designation"]], how="left", on="DAC_ID"
)

##### Match remaining unaccounted for streets with geo-spacial join

In [ ]:
# Most DAC tracts match to our Geo_IDs but a few of them dont match perfectly probably for some census tract details I cant perfectly answer.
# Due to this small issue with matching DAC_IDs to Geo_IDs we match the remaining blocks to their nearest DAC/NotDAC census tract based on geography.
# Note that this step is also an approximation and could be improved with some geographical research

DAC_near = (
    lion_walkable[lion_walkable["DAC_Designation"].isna()]
    .loc[:, ["uniqueID", "geometry"]]
    .sjoin_nearest(DAC_shapes, how="left", distance_col="dis")
)
DAC_near = DAC_near.sort_values("DAC_Designation", ascending=False).sort_values(
    "dis", ascending=False
)
DAC_near = DAC_near.groupby("uniqueID").first().reset_index()
DAC_near = DAC_near[["uniqueID", "DAC_ID", "DAC_Designation"]].rename(
    columns={"DAC_ID": "DAC_ID_geom", "DAC_Designation": "DAC_Designation_geom"}
)

lion_walkable = lion_walkable.merge(DAC_near, on="uniqueID", how="left")
lion_walkable.loc[lion_walkable["DAC_Designation"].isna(), "DAC_ID"] = (
    lion_walkable.loc[lion_walkable["DAC_Designation"].isna(), "DAC_ID_geom"]
)
lion_walkable.loc[lion_walkable["DAC_Designation"].isna(), "DAC_Designation"] = (
    lion_walkable.loc[lion_walkable["DAC_Designation"].isna(), "DAC_Designation_geom"]
)
lion_walkable = lion_walkable.drop(columns=["DAC_ID_geom", "DAC_Designation_geom"])

#### Assign lion streets to Gun Violence Prevention police precincts

In [ ]:
# Import and format Police Precinct boundaries

url = """https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Police_Precincts/FeatureServer/0/query?where=1%3D1&objectIds=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=&returnGeometry=true&returnCentroid=false&returnEnvelope=false&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&collation=&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="""

precinct_shapes = gpd.read_file(url)
precinct_shapes = precinct_shapes[["Precinct", "geometry"]].to_crs(2263)
precinct_shapes.loc[
    precinct_shapes["Precinct"].isin([40, 42, 44, 47, 73, 75]), "GVP"
] = "Yes GVP"
precinct_shapes.loc[
    ~precinct_shapes["Precinct"].isin([40, 42, 44, 47, 73, 75]), "GVP"
] = "No GVP"

In [ ]:
# Assign blocks to their police precinct by geography. This is an approximation that could be improved with geographical research.

precinct_near = lion_walkable.loc[:, ["uniqueID", "geometry"]].sjoin_nearest(
    precinct_shapes, how="left", distance_col="dis"
)
precinct_near = precinct_near.sort_values("Precinct", ascending=False).sort_values(
    "dis", ascending=False
)
precinct_near = precinct_near.groupby("uniqueID").first().reset_index()
precinct_near = precinct_near[["uniqueID", "Precinct", "GVP"]]

lion_walkable = lion_walkable.merge(precinct_near, on="uniqueID", how="left")

### Match Lion Data to TRIE communities

In [ ]:
# Import and format TRIE community boundaries.

url = """https://services3.arcgis.com/xJHn8F2NTtwCMFtX/ArcGIS/rest/services/TRIE/FeatureServer/0/query"""

params = {"where": "1=1", "outfields": "*", "f": "json"}
response = requests.get(url, params=params)
Trie_shapes = response.json()
Trie_shapes = pd.json_normalize(Trie_shapes["features"])
for row in Trie_shapes.index:
    Trie_shapes.loc[row, "geometry"] = Polygon(
        Trie_shapes.loc[row, "geometry.rings"][0]
    )
Trie_shapes = Trie_shapes[
    ["attributes.OBJECTID", "attributes.Neighborhood", "geometry"]
].rename(
    columns={
        "attributes.OBJECTID": "Trie_label",
        "attributes.Neighborhood": "Trie_name",
    }
)
Trie_shapes = gpd.GeoDataFrame(Trie_shapes, geometry="geometry", crs=2263)

In [ ]:
# Assign city blocks to TRIE communities based on geographical approximations. Perhaps can be improved with more geographical research.

trie_intersections = gpd.overlay(
    lion_walkable[["geometry", "uniqueID", "length"]], Trie_shapes
)
trie_intersections["len"] = trie_intersections.length
trie_intersections = trie_intersections.sort_values("len", ascending=False)
trie_intersections = trie_intersections.groupby("uniqueID").first().reset_index()
trie_intersections.loc[
    trie_intersections["len"] < (0.3 * trie_intersections["length"]), "Trie_label"
] = 0
trie_intersections.loc[
    trie_intersections["len"] < (0.3 * trie_intersections["length"]), "Trie_name"
] = "Not Trie"
trie_intersections = trie_intersections[["uniqueID", "Trie_label", "Trie_name"]]

lion_walkable = lion_walkable.merge(trie_intersections, on="uniqueID", how="left")
lion_walkable.loc[lion_walkable["Trie_label"].isna(), "Trie_label"] = 0
lion_walkable.loc[lion_walkable["Trie_name"].isna(), "Trie_name"] = "Not Trie"

## Update Walkable_DG graph with full lion_walkable atributes

In [ ]:
# Create NetworkX graph based on these city-block edges and store each blocks geographical and population data as edge attributes

Walk_DG = nx.from_pandas_edgelist(
    lion_walkable,
    create_using=nx.DiGraph(),
    edge_attr=[
        "uniqueID",
        "Street",
        "length",
        "geometry",
        "SegmentID",
        "XCoord_source",
        "YCoord_source",
        "XCoord_target",
        "YCoord_target",
        "RW_TYPE",
        "FeatureTyp",
        "NYPDID",
        "SideOfStreet",
        "From",
        "To",
        "Boro",
        "CommunityBoard",
        "weight",
        "mode",
        "CT2020",
        "CT2020Suf",
        "CB2020",
        "CB2020Suf",
        "Geo_ID",
        "livable_length",
        "Total_geo_ID_livable_length",
        "Proportion_of_blockgroup",
        "BG_total_pop",
        "population",
        "DAC_ID",
        "DAC_Designation",
        "Precinct",
        "GVP",
        "Trie_label",
        "Trie_name",
        "Council_District",
    ],
)

In [ ]:
# Also store geographic location information (lat/lon) for all of our nodes (segment connections)

lion_walkable_nodes = pd.concat(
    [
        lion_walkable[["source", "XCoord_source", "YCoord_source"]].rename(
            columns={"XCoord_source": "x", "YCoord_source": "y", "source": "NodeID"}
        ),
        lion_walkable[["target", "XCoord_target", "YCoord_target"]].rename(
            columns={"XCoord_target": "x", "YCoord_target": "y", "target": "NodeID"}
        ),
    ]
)
lion_walkable_nodes = (
    lion_walkable_nodes.drop_duplicates(subset="NodeID")
    .set_index("NodeID")
    .apply(lambda x: Point(x["x"], x["y"]), axis=1)
)

nx.set_node_attributes(Walk_DG, lion_walkable_nodes.to_dict(), name="geometry")

## Prepare Transit network

### Import stops and stoptime gtfs data for MTA subways and buses

In [ ]:
# We include city bus, special bus, and subway data. LIRR and other transit data can also be found on the MTA developer page if one is interested
bx_bus_stops, bx_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_bx.zip"""
)
bk_bus_stops, bk_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_b.zip"""
)
mn_bus_stops, mn_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_m.zip"""
)
qn_bus_stops, qn_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_q.zip"""
)
si_bus_stops, si_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_si.zip"""
)
exp_bus_stops, exp_bus_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_busco.zip"""
)
subway_stops, subway_stop_times = gtfs_mta_import(
    """https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip"""
)

### Format stop locations

In [ ]:
url = """https://services5.arcgis.com/GfwWNkhOj9bNBqoJ/arcgis/rest/services/NYC_Borough_Boundary_Water_Included/FeatureServer/0/query?where=1=1&outFields=*&outSR=4326&f=pgeojson"""
response = requests.get(url)
boros = gpd.GeoDataFrame.from_features(response.json()).set_crs(4326).to_crs(2263)

In [ ]:
# Format stops data
stops = pd.concat(
    [
        bx_bus_stops,
        bk_bus_stops,
        mn_bus_stops,
        qn_bus_stops,
        si_bus_stops,
        exp_bus_stops,
        subway_stops,
    ]
)
stops = format_mta_stops_data(stops)

# Only include stops within the city limits
city = (
    gpd.GeoDataFrame([boros.buffer(100).unary_union])
    .rename(columns={0: "geometry"})
    .set_geometry(col="geometry", crs=2263)
)
stops = gpd.sjoin(stops, city, how="left")
stops = stops.loc[~stops["index_right"].isna(), ["stop_id", "stop_name", "geometry"]]

In [ ]:
# Add edges that connect each stop to its nearest level M (ground level) street node.
# Add a weight for how long it takes to walk from street node to stop node or vice versa.

lion_street_nodes_gdf = (
    gpd.GeoDataFrame(lion_walkable_nodes)
    .reset_index()
    .rename(columns={0: "geometry"})
    .set_geometry(col="geometry", crs=2263)
)
lion_street_nodes_gdf = lion_street_nodes_gdf[
    lion_street_nodes_gdf["NodeID"].str[-1] == "M"
]
stop_node_connections = gpd.sjoin_nearest(
    stops, lion_street_nodes_gdf, how="left", distance_col="dist"
)
stop_node_connections = stop_node_connections.sort_values("dist")
stop_node_connections = (
    stop_node_connections.groupby(["stop_id", "NodeID"])
    .agg({"stop_name": "first", "dist": "min"})
    .reset_index()
)

In [ ]:
# Make sure we can both enter and exit every stop by swapping the source and target and concatenating the resulting dataframes

stop_node_connections["stop_id"] = stop_node_connections["stop_id"] + "_mta"
stop_node_connections["mode"] = "walk_mta_connection"
stop_node_connections["weight"] = (
    stop_node_connections["dist"] / walk_speed_feet_per_second
)
stop_node_connections_exit = stop_node_connections.rename(
    columns={"stop_id": "source", "NodeID": "target"}
)
stop_node_connections_enter = stop_node_connections.rename(
    columns={"stop_id": "target", "NodeID": "source"}
)
stop_node_connections = pd.concat(
    [stop_node_connections_enter, stop_node_connections_exit]
)

# Create a networkX network made of of edges that connect each MTA stop to its nearest street level node
walk_to_MTA_DG = nx.from_pandas_edgelist(
    stop_node_connections,
    create_using=nx.DiGraph(),
    edge_attr=["stop_name", "weight", "mode"],
)

# Stitch the walkable street network and the enter/exit MTA stop locations network together
complete_DG = nx.compose(Walk_DG, walk_to_MTA_DG)

### Calculate travel times (including expected wait times) between connected stops

This code chunk calculates the average travel time between any connected pair of MTA stops.

This average is with respect to the times specified by our initial parameters:
- weekend_transit (True/False), transit_start_hr (INT btwn 0-24)
- transit_end_hr (INT btwn transit_start_hr-24)

Intuitively, we are calculating the average fastest possible travel time between each feasible stop-pair where this average is over all possible trip start times that fall between our set transit_start_hr and transit_end_hr parameters.

In [ ]:
# Format stop-times data
stop_times = pd.concat(
    [
        bx_bus_stop_times,
        bk_bus_stop_times,
        mn_bus_stop_times,
        qn_bus_stop_times,
        si_bus_stop_times,
        exp_bus_stop_times,
        subway_stop_times,
    ]
)
stop_times["stop_id"] = stop_times["stop_id"].astype(str)
stop_times["stop_id"] = stop_times["stop_id"] + "_mta"

# Remove any stops that are not part of our walkable-street to MTA stop connected network. (e.g. not in city boundaries like Hoboken...)
stop_times = stop_times[
    stop_times["stop_id"].isin(stop_node_connections_exit["source"].unique())
]

# If weekend_transit is set to false then only include weekday trips
if weekend_transit == False:
    trips = stop_times[stop_times["trip_id"].str.contains("Weekday")]

# Only consider trips between our chosen start and end times
trips = trips.loc[
    (trips["departure_time"] >= str(transit_start_hr).zfill(2))
    & (trips["departure_time"] <= str(transit_end_hr).zfill(2))
]

Create the collection of pairwise stop-to-stop connections.

We take this more detailed approach of stop-to-stop edges instead of the previous route based approach because we want to account for the 14th to w4 situation.

In this situation you would happily take either the A, C, or E train, which ever comes first.

So when you arrive a at 14th street you are not waiting for any MTA route one but for the first of three possible MTA routes that arrives.

In [ ]:
# Connect any two stops that are part of a shared route.
departures = trips[["trip_id", "departure_time", "stop_id", "stop_sequence"]].rename(
    columns={"stop_id": "departure_stop", "stop_sequence": "dep_sequence"}
)
arrivals = trips[["trip_id", "arrival_time", "stop_id", "stop_sequence"]].rename(
    columns={"stop_id": "arrival_stop", "stop_sequence": "arr_sequence"}
)
stop_to_stop = departures.merge(arrivals, on="trip_id", how="left")

In [ ]:
# Stop connections are directed so only include the situations where the departure stop comes before the arrival stop in that route
stop_to_stop = stop_to_stop[stop_to_stop["dep_sequence"] < stop_to_stop["arr_sequence"]]

# Provide a unique label for each feasible departure_stop-to-arrival_stop pairing
stop_to_stop["stop_pair"] = (
    stop_to_stop["departure_stop"].astype(str)
    + "_"
    + stop_to_stop["arrival_stop"].astype(str)
)
stop_to_stop = stop_to_stop[
    [
        "trip_id",
        "departure_time",
        "departure_stop",
        "arrival_time",
        "arrival_stop",
        "stop_pair",
    ]
].reset_index(drop=True)

In [ ]:
# There are some stop to stop connections one should never take such as riding a local train to get between far apart express stops.
# We remove these artificially slower connections before proceeding to calculate the average stop-to-stop travel times.
dep_arr_ranks = (
    stop_to_stop.groupby("stop_pair")[["departure_time", "arrival_time"]]
    .rank("min")
    .rename(
        columns={"departure_time": "sts_depart_rank", "arrival_time": "sts_arrive_rank"}
    )
)
stop_to_stop["sts_depart_rank"] = dep_arr_ranks["sts_depart_rank"]
stop_to_stop["sts_arrive_rank"] = dep_arr_ranks["sts_arrive_rank"]
stop_to_stop = stop_to_stop.sort_values("sts_arrive_rank").reset_index(drop=True)
drop_trips = stop_to_stop.groupby("stop_pair")["sts_depart_rank"].diff()
while (drop_trips <= 0).sum() > 0:
    stop_to_stop = stop_to_stop[~(drop_trips <= 0)]
    drop_trips = stop_to_stop.groupby("stop_pair")["sts_depart_rank"].diff()

In [ ]:
# Here we calculate the expected wait time for each possible stop-to-stop trip.
# This is equal to half the time between the last feasible departure and the next feasible departure for each stop-to-stop pair.
stop_to_stop = stop_to_stop.reset_index(drop=True)
stop_to_stop = stop_to_stop[
    [
        "departure_time",
        "departure_stop",
        "arrival_time",
        "arrival_stop",
        "stop_pair",
        "sts_depart_rank",
    ]
]
stop_to_stop["sts_depart_rank"] = stop_to_stop.groupby("stop_pair")[
    "departure_time"
].rank()
previous_depart = stop_to_stop[
    ["stop_pair", "sts_depart_rank", "departure_time"]
].rename(columns={"departure_time": "prev_departure_time"})
previous_depart["sts_depart_rank"] = previous_depart["sts_depart_rank"] + 1
stop_to_stop = stop_to_stop.merge(
    previous_depart, on=["stop_pair", "sts_depart_rank"], how="left"
)
stop_to_stop.loc[stop_to_stop["prev_departure_time"].isna(), "prev_departure_time"] = (
    str(transit_start_hr).zfill(2) + ":00:00"
)
stop_to_stop["departure_time"] = pd.to_datetime(
    stop_to_stop["departure_time"], format="%H:%M:%S"
)
stop_to_stop["arrival_time"] = pd.to_datetime(
    stop_to_stop["arrival_time"], format="%H:%M:%S"
)
stop_to_stop["prev_departure_time"] = pd.to_datetime(
    stop_to_stop["prev_departure_time"], format="%H:%M:%S"
)
stop_to_stop["wait_time_secs"] = (
    stop_to_stop["departure_time"] - stop_to_stop["prev_departure_time"]
).dt.total_seconds() / 2

# Here we calculate the actual in transit time for each trip
stop_to_stop["travel_time_secs"] = (
    stop_to_stop["arrival_time"] - stop_to_stop["departure_time"]
).dt.total_seconds()

# Here we add the expected wait time and the actual transit time for each trip to get the total travel time for each trip
stop_to_stop["total_transit_time_secs"] = (
    stop_to_stop["wait_time_secs"] + stop_to_stop["travel_time_secs"]
)

In [ ]:
# Now that we have calculated the total travel time for each possible stop-to-stop trip
# we compute the average travel time for every feasible stop-to-stop pair during our parameter defined travel window
stop_to_stop = (
    stop_to_stop.groupby("stop_pair")
    .agg(
        {
            "departure_stop": "first",
            "arrival_stop": "first",
            "total_transit_time_secs": "median",
        }
    )
    .reset_index()
)

# We format these transit connections to match the source/target convention used by networkX
stop_to_stop["mode"] = "mta"
stop_to_stop = stop_to_stop[
    ["departure_stop", "arrival_stop", "mode", "total_transit_time_secs"]
].rename(
    columns={
        "departure_stop": "source",
        "arrival_stop": "target",
        "total_transit_time_secs": "weight",
    }
)

In [ ]:
# We create the directed network of stop-to-stop mta connections where the weight of each edge is the average travel time (in seconds) for that stop-to-stop pair

MTA_DG = nx.from_pandas_edgelist(
    stop_to_stop, create_using=nx.DiGraph(), edge_attr=["weight", "mode"]
)

In [ ]:
# We stitch this directed transit network together with our other network of both walkable streets and (MTA stops)-to-(walkable streets) connections

complete_DG = nx.compose(complete_DG, MTA_DG)

# We have now finished creating our networkX walking and MTA NYC network.

# Below we choose to store the parts of this network we want to keep as geodataframes

In [ ]:
# Here we store the entire network edgeset in one geodataframe which we will use in future vital parks scripts

Vital_Parks_Edges = nx.to_pandas_edgelist(complete_DG)
Vital_Parks_Edges = gpd.GeoDataFrame(Vital_Parks_Edges, geometry="geometry").set_crs(
    2263
)

In [ ]:
# Here we store the street-level nodes in a geodataframe because we want to keep the geo-location data for these particular nodes for the remaining Vital Parks Scripts

Vital_Parks_Street_Nodes = (
    pd.DataFrame([nx.get_node_attributes(complete_DG, "geometry")])
    .transpose()
    .reset_index()
    .rename(columns={0: "geometry", "index": "street_node_id"})
)
Vital_Parks_Street_Nodes = gpd.GeoDataFrame(
    Vital_Parks_Street_Nodes, geometry="geometry"
).set_crs(2263)

# Restrict this set of street points so that we can only access the network at ground level (Level 'M').
# This prevents us, for example, from 'entering' the street network in the middle of a bridge. We would only include the entrance nodes for this bridge and none of the higher midpoints.
Vital_Parks_Street_Nodes = Vital_Parks_Street_Nodes[
    Vital_Parks_Street_Nodes["street_node_id"].str.contains("M")
].reset_index(drop=True)

In [ ]:
# ##################################################
# Upload/save these edges and nodes geodataframes to SQL server, as a GeoJSON, or however you want to store it so that they can be accessed by the third vital parks script
# ##################################################

Vital_Parks_Edges.to_file("data/Network_edges.geojson", driver="GeoJSON")

Vital_Parks_Street_Nodes.to_file("data/Network_nodes.geojson", driver="GeoJSON")

# END Vital Parks Script Part 1